# The notebook contains
### Code for _Bulyan_ aggregation algorithm
### Evaluation of our SOTA AGR-tailored attack on Bulyan

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import json
from __future__ import print_function
import argparse, os, sys, csv, shutil, time, random, operator, pickle, ast, math
import numpy as np
import pandas as pd
from torch.optim import Optimizer
import torch.nn.functional as F
import torch
import pickle
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torch.multiprocessing as mp

sys.path.insert(0,'/home/vshejwalkar/fed-quant-robustness/code/utils/')
from logger import *
from eval import *
from misc import *

from femnist_normal_train import *
from femnist_util import *
from adam import Adam
from sgd import SGD
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## Get the FEMNIST dataset; we use [LEAF framework](https://leaf.cmu.edu/)

In [5]:
user_tr_data = []
user_tr_labels = []

for i in range(34):
    f = '/mnt/nfs/work1/amir/vshejwalkar/leaf/data/femnist/data/train/all_data_%d_niid_0_keep_0_train_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_tr_data.append(obj['user_data'][user]['x'])
        user_tr_labels.append(obj['user_data'][user]['y'])

user_te_data = []
user_te_labels = []

for i in range(34):
    f = '/mnt/nfs/work1/amir/vshejwalkar/leaf/data/femnist/data/test/all_data_%d_niid_0_keep_0_test_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_te_data.append(obj['user_data'][user]['x'])
        user_te_labels.append(obj['user_data'][user]['y'])

In [6]:
user_tr_data_tensors=[]
user_tr_label_tensors=[]

for i in range(len(user_tr_data)):
    
    user_tr_data_tensor=torch.from_numpy(np.array(user_tr_data[i])).type(torch.FloatTensor)
    user_tr_label_tensor=torch.from_numpy(np.array(user_tr_labels[i])).type(torch.LongTensor)

    user_tr_data_tensors.append(user_tr_data_tensor)
    user_tr_label_tensors.append(user_tr_label_tensor)
    
    print('user %d tr len %d'%(i,len(user_tr_data_tensor)))

user 0 tr len 333
user 1 tr len 310
user 2 tr len 375
user 3 tr len 344
user 4 tr len 319
user 5 tr len 218
user 6 tr len 119
user 7 tr len 275
user 8 tr len 270
user 9 tr len 377
user 10 tr len 335
user 11 tr len 346
user 12 tr len 327
user 13 tr len 358
user 14 tr len 337
user 15 tr len 354
user 16 tr len 369
user 17 tr len 367
user 18 tr len 276
user 19 tr len 299
user 20 tr len 348
user 21 tr len 342
user 22 tr len 334
user 23 tr len 378
user 24 tr len 333
user 25 tr len 390
user 26 tr len 160
user 27 tr len 351
user 28 tr len 363
user 29 tr len 378
user 30 tr len 337
user 31 tr len 354
user 32 tr len 267
user 33 tr len 398
user 34 tr len 289
user 35 tr len 351
user 36 tr len 387
user 37 tr len 390
user 38 tr len 225
user 39 tr len 344
user 40 tr len 323
user 41 tr len 377
user 42 tr len 292
user 43 tr len 324
user 44 tr len 185
user 45 tr len 376
user 46 tr len 225
user 47 tr len 373
user 48 tr len 360
user 49 tr len 385
user 50 tr len 324
user 51 tr len 321
user 52 tr len 353
use

user 428 tr len 324
user 429 tr len 333
user 430 tr len 141
user 431 tr len 338
user 432 tr len 276
user 433 tr len 345
user 434 tr len 248
user 435 tr len 314
user 436 tr len 204
user 437 tr len 293
user 438 tr len 261
user 439 tr len 372
user 440 tr len 194
user 441 tr len 344
user 442 tr len 312
user 443 tr len 274
user 444 tr len 199
user 445 tr len 367
user 446 tr len 368
user 447 tr len 209
user 448 tr len 342
user 449 tr len 271
user 450 tr len 271
user 451 tr len 336
user 452 tr len 297
user 453 tr len 321
user 454 tr len 393
user 455 tr len 314
user 456 tr len 238
user 457 tr len 287
user 458 tr len 374
user 459 tr len 234
user 460 tr len 313
user 461 tr len 217
user 462 tr len 301
user 463 tr len 353
user 464 tr len 355
user 465 tr len 362
user 466 tr len 377
user 467 tr len 344
user 468 tr len 306
user 469 tr len 264
user 470 tr len 273
user 471 tr len 273
user 472 tr len 343
user 473 tr len 352
user 474 tr len 292
user 475 tr len 235
user 476 tr len 283
user 477 tr len 342


user 845 tr len 126
user 846 tr len 163
user 847 tr len 135
user 848 tr len 159
user 849 tr len 140
user 850 tr len 160
user 851 tr len 158
user 852 tr len 155
user 853 tr len 156
user 854 tr len 160
user 855 tr len 152
user 856 tr len 152
user 857 tr len 143
user 858 tr len 155
user 859 tr len 142
user 860 tr len 160
user 861 tr len 155
user 862 tr len 157
user 863 tr len 157
user 864 tr len 158
user 865 tr len 154
user 866 tr len 162
user 867 tr len 122
user 868 tr len 152
user 869 tr len 150
user 870 tr len 157
user 871 tr len 161
user 872 tr len 164
user 873 tr len 135
user 874 tr len 148
user 875 tr len 142
user 876 tr len 158
user 877 tr len 135
user 878 tr len 156
user 879 tr len 161
user 880 tr len 151
user 881 tr len 151
user 882 tr len 154
user 883 tr len 164
user 884 tr len 153
user 885 tr len 164
user 886 tr len 136
user 887 tr len 152
user 888 tr len 146
user 889 tr len 153
user 890 tr len 152
user 891 tr len 162
user 892 tr len 85
user 893 tr len 153
user 894 tr len 162
u

user 1246 tr len 144
user 1247 tr len 93
user 1248 tr len 144
user 1249 tr len 145
user 1250 tr len 120
user 1251 tr len 151
user 1252 tr len 157
user 1253 tr len 159
user 1254 tr len 97
user 1255 tr len 161
user 1256 tr len 139
user 1257 tr len 149
user 1258 tr len 157
user 1259 tr len 123
user 1260 tr len 128
user 1261 tr len 153
user 1262 tr len 104
user 1263 tr len 133
user 1264 tr len 122
user 1265 tr len 147
user 1266 tr len 138
user 1267 tr len 154
user 1268 tr len 149
user 1269 tr len 90
user 1270 tr len 127
user 1271 tr len 111
user 1272 tr len 155
user 1273 tr len 161
user 1274 tr len 158
user 1275 tr len 88
user 1276 tr len 138
user 1277 tr len 151
user 1278 tr len 78
user 1279 tr len 147
user 1280 tr len 94
user 1281 tr len 135
user 1282 tr len 145
user 1283 tr len 143
user 1284 tr len 149
user 1285 tr len 37
user 1286 tr len 156
user 1287 tr len 151
user 1288 tr len 144
user 1289 tr len 97
user 1290 tr len 123
user 1291 tr len 144
user 1292 tr len 148
user 1293 tr len 161


user 1661 tr len 154
user 1662 tr len 155
user 1663 tr len 158
user 1664 tr len 162
user 1665 tr len 160
user 1666 tr len 129
user 1667 tr len 148
user 1668 tr len 153
user 1669 tr len 162
user 1670 tr len 162
user 1671 tr len 145
user 1672 tr len 157
user 1673 tr len 142
user 1674 tr len 154
user 1675 tr len 141
user 1676 tr len 150
user 1677 tr len 153
user 1678 tr len 149
user 1679 tr len 145
user 1680 tr len 153
user 1681 tr len 155
user 1682 tr len 157
user 1683 tr len 156
user 1684 tr len 142
user 1685 tr len 161
user 1686 tr len 155
user 1687 tr len 134
user 1688 tr len 157
user 1689 tr len 164
user 1690 tr len 152
user 1691 tr len 148
user 1692 tr len 147
user 1693 tr len 160
user 1694 tr len 158
user 1695 tr len 153
user 1696 tr len 159
user 1697 tr len 157
user 1698 tr len 161
user 1699 tr len 161
user 1700 tr len 156
user 1701 tr len 153
user 1702 tr len 143
user 1703 tr len 148
user 1704 tr len 153
user 1705 tr len 160
user 1706 tr len 163
user 1707 tr len 160
user 1708 tr 

user 2074 tr len 153
user 2075 tr len 140
user 2076 tr len 150
user 2077 tr len 142
user 2078 tr len 162
user 2079 tr len 127
user 2080 tr len 160
user 2081 tr len 164
user 2082 tr len 151
user 2083 tr len 158
user 2084 tr len 152
user 2085 tr len 147
user 2086 tr len 134
user 2087 tr len 148
user 2088 tr len 131
user 2089 tr len 153
user 2090 tr len 152
user 2091 tr len 145
user 2092 tr len 158
user 2093 tr len 153
user 2094 tr len 161
user 2095 tr len 159
user 2096 tr len 158
user 2097 tr len 141
user 2098 tr len 127
user 2099 tr len 161
user 2100 tr len 162
user 2101 tr len 147
user 2102 tr len 162
user 2103 tr len 160
user 2104 tr len 163
user 2105 tr len 167
user 2106 tr len 159
user 2107 tr len 160
user 2108 tr len 164
user 2109 tr len 162
user 2110 tr len 153
user 2111 tr len 153
user 2112 tr len 160
user 2113 tr len 160
user 2114 tr len 153
user 2115 tr len 160
user 2116 tr len 164
user 2117 tr len 153
user 2118 tr len 160
user 2119 tr len 160
user 2120 tr len 162
user 2121 tr 

user 2475 tr len 164
user 2476 tr len 161
user 2477 tr len 163
user 2478 tr len 140
user 2479 tr len 166
user 2480 tr len 164
user 2481 tr len 153
user 2482 tr len 144
user 2483 tr len 164
user 2484 tr len 143
user 2485 tr len 136
user 2486 tr len 165
user 2487 tr len 165
user 2488 tr len 137
user 2489 tr len 160
user 2490 tr len 146
user 2491 tr len 153
user 2492 tr len 147
user 2493 tr len 154
user 2494 tr len 156
user 2495 tr len 155
user 2496 tr len 162
user 2497 tr len 142
user 2498 tr len 154
user 2499 tr len 158
user 2500 tr len 151
user 2501 tr len 157
user 2502 tr len 164
user 2503 tr len 162
user 2504 tr len 141
user 2505 tr len 164
user 2506 tr len 159
user 2507 tr len 155
user 2508 tr len 149
user 2509 tr len 128
user 2510 tr len 162
user 2511 tr len 159
user 2512 tr len 146
user 2513 tr len 161
user 2514 tr len 135
user 2515 tr len 160
user 2516 tr len 154
user 2517 tr len 160
user 2518 tr len 146
user 2519 tr len 141
user 2520 tr len 155
user 2521 tr len 147
user 2522 tr 

user 2872 tr len 297
user 2873 tr len 260
user 2874 tr len 312
user 2875 tr len 360
user 2876 tr len 215
user 2877 tr len 284
user 2878 tr len 254
user 2879 tr len 277
user 2880 tr len 354
user 2881 tr len 289
user 2882 tr len 285
user 2883 tr len 333
user 2884 tr len 352
user 2885 tr len 231
user 2886 tr len 226
user 2887 tr len 298
user 2888 tr len 375
user 2889 tr len 353
user 2890 tr len 337
user 2891 tr len 288
user 2892 tr len 354
user 2893 tr len 363
user 2894 tr len 303
user 2895 tr len 264
user 2896 tr len 351
user 2897 tr len 275
user 2898 tr len 306
user 2899 tr len 222
user 2900 tr len 353
user 2901 tr len 320
user 2902 tr len 223
user 2903 tr len 333
user 2904 tr len 311
user 2905 tr len 349
user 2906 tr len 334
user 2907 tr len 297
user 2908 tr len 241
user 2909 tr len 323
user 2910 tr len 341
user 2911 tr len 306
user 2912 tr len 349
user 2913 tr len 300
user 2914 tr len 342
user 2915 tr len 262
user 2916 tr len 257
user 2917 tr len 265
user 2918 tr len 223
user 2919 tr 

user 3271 tr len 230
user 3272 tr len 260
user 3273 tr len 221
user 3274 tr len 270
user 3275 tr len 186
user 3276 tr len 297
user 3277 tr len 134
user 3278 tr len 198
user 3279 tr len 199
user 3280 tr len 243
user 3281 tr len 342
user 3282 tr len 153
user 3283 tr len 184
user 3284 tr len 209
user 3285 tr len 252
user 3286 tr len 283
user 3287 tr len 233
user 3288 tr len 235
user 3289 tr len 162
user 3290 tr len 297
user 3291 tr len 257
user 3292 tr len 208
user 3293 tr len 252
user 3294 tr len 229
user 3295 tr len 322
user 3296 tr len 254
user 3297 tr len 209
user 3298 tr len 242
user 3299 tr len 328
user 3300 tr len 296
user 3301 tr len 339
user 3302 tr len 294
user 3303 tr len 224
user 3304 tr len 243
user 3305 tr len 253
user 3306 tr len 253
user 3307 tr len 222
user 3308 tr len 225
user 3309 tr len 214
user 3310 tr len 268
user 3311 tr len 294
user 3312 tr len 327
user 3313 tr len 108
user 3314 tr len 317
user 3315 tr len 186
user 3316 tr len 160
user 3317 tr len 313
user 3318 tr 

In [7]:
te_data = np.concatenate(user_te_data, 0)
te_labels = np.concatenate(user_te_labels)
te_len = len(te_labels)

te_data_tensor = torch.from_numpy(te_data[:(te_len//2)]).type(torch.FloatTensor)
te_label_tensor = torch.from_numpy(te_labels[:(te_len//2)]).type(torch.LongTensor)

val_data_tensor = torch.from_numpy(te_data[(te_len//2):]).type(torch.FloatTensor)
val_label_tensor = torch.from_numpy(te_labels[(te_len//2):]).type(torch.LongTensor)

## Code for Bulyan aggregation algorithm

In [9]:
def bulyan(all_updates, n_attackers):
    nusers = all_updates.shape[0]
    bulyan_cluster = []
    candidate_indices = []
    remaining_updates = all_updates
    all_indices = np.arange(len(all_updates))

    while len(bulyan_cluster) < (nusers - 2 * n_attackers):
        distances = []
        for update in remaining_updates:
            distance = torch.norm((remaining_updates - update), dim=1) ** 2
            distances = distance[None, :] if not len(distances) else torch.cat((distances, distance[None, :]), 0)

        distances = torch.sort(distances, dim=1)[0]

        scores = torch.sum(distances[:, :len(remaining_updates) - 2 - n_attackers], dim=1)
        indices = torch.argsort(scores)[:len(remaining_updates) - 2 - n_attackers]

        candidate_indices.append(all_indices[indices[0].cpu().numpy()])
        all_indices = np.delete(all_indices, indices[0].cpu().numpy())
        bulyan_cluster = remaining_updates[indices[0]][None, :] if not len(bulyan_cluster) else torch.cat((bulyan_cluster, remaining_updates[indices[0]][None, :]), 0)
        remaining_updates = torch.cat((remaining_updates[:indices[0]], remaining_updates[indices[0] + 1:]), 0)

    # print('dim of bulyan cluster ', bulyan_cluster.shape)

    n, d = bulyan_cluster.shape
    param_med = torch.median(bulyan_cluster, dim=0)[0]
    sort_idx = torch.argsort(torch.abs(bulyan_cluster - param_med), dim=0)
    sorted_params = bulyan_cluster[sort_idx, torch.arange(d)[None, :]]

    return torch.mean(sorted_params[:n - 2 * n_attackers], dim=0), np.array(candidate_indices)

## Code for Fang attack on Bulyan
### Fang attacks on MKrum and Bulyan are the same

In [8]:
def our_attack_mkrum(all_updates, model_re, n_attackers,dev_type='unit_vec'):

    if dev_type == 'unit_vec':
        deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
    elif dev_type == 'sign':
        deviation = torch.sign(model_re)
    elif dev_type == 'std':
        deviation = torch.std(all_updates, 0)
        
    lamda = torch.Tensor([20.0]).cuda() #compute_lambda_our(all_updates, model_re, n_attackers)
    # print(lamda)
    threshold_diff = 1e-5
    lamda_fail = lamda
    lamda_succ = 0

    while torch.abs(lamda_succ - lamda) > threshold_diff:
        mal_update = (model_re - lamda * deviation)
        mal_updates = torch.stack([mal_update] * n_attackers)
        mal_updates = torch.cat((mal_updates, all_updates), 0)

        agg_grads, krum_candidate = multi_krum(mal_updates, n_attackers, multi_k=True)

        if np.sum(krum_candidate < n_attackers) == n_attackers:
            # print('successful lamda is ', lamda)
            lamda_succ = lamda
            lamda = lamda + lamda_fail / 2
        else:
            lamda = lamda - lamda_fail / 2

        lamda_fail = lamda_fail / 2

    mal_update = (model_re - lamda_succ * deviation)

    return mal_update

In [11]:
resume=0
nepochs=1500
gamma=.1
fed_lr=0.001

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()
batch_size = 100
schedule = [2000]

aggregation = 'bulyan'
at_type = 'our-agr'
chkpt = './' + aggregation
epoch_num = 0

at_fractions = [20]

for at_fraction in at_fractions:

    fed_model = mnist_conv().cuda()
    fed_model.apply(weights_init)
    optimizer_fed = Adam(fed_model.parameters(), lr=fed_lr)

    print('==> Initializing global model')
    epoch_num = 0
    best_global_acc=0
    best_global_te_acc=0

    while epoch_num <= nepochs:
        user_grads = []

        round_users = np.random.choice(3400, 60)
        n_attacker = max(2, np.sum(round_users < (34*at_fraction)))
        if n_attacker > 14:
            # print ('n_attackers actual %d adjusted 14' % n_attacker)
            n_attacker = 14

        # print('n_attackers is ', n_attackers)
        attacker_count = 0
        for i in round_users:
            if i < (34*at_fraction) and attacker_count < n_attacker:
                attacker_count += 1
                continue

            inputs = user_tr_data_tensors[i]
            targets = user_tr_label_tensors[i]

            inputs, targets = inputs.cuda(), targets.cuda()
            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

            outputs = fed_model(inputs)
            loss = criterion(outputs, targets)
            optimizer_fed.zero_grad()
            loss.backward(retain_graph=True)

            param_grad=[]
            for param in fed_model.parameters():
                param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))

            user_grads=param_grad[None,:] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]),0)    

        malicious_grads = user_grads

        if n_attacker > 0:
            if at_type == 'fang':
                agg_grads = torch.mean(malicious_grads, 0)
                deviation = torch.sign(agg_grads)
                mal_update = get_malicious_updates_fang(malicious_grads, agg_grads, deviation, n_attacker)

            elif at_type == 'our-agr':
                agg_grads = torch.mean(malicious_grads, 0)
                mal_update = our_attack_mkrum(malicious_grads, agg_grads, n_attacker, dev_type='sign')
            
            mal_updates = torch.stack([mal_update] * n_attacker)
            malicious_grads = torch.cat((mal_updates, user_grads), 0)    
        
        if epoch_num == 0: print('malicious grads shape ', malicious_grads.shape)

        if aggregation == 'mean':
            agg_grads=torch.mean(malicious_grads,dim=0)
            
        elif aggregation=='krum' or aggregation=='mkrum':
            multi_k = True if aggregation == 'mkrum' else False
            if epoch_num == 0: print('multi krum is ', multi_k)
            agg_grads, krum_candidate = multi_krum(malicious_grads, n_attacker, multi_k=multi_k)

        elif aggregation == 'bulyan':
            agg_grads, krum_candidate = bulyan(malicious_grads, n_attacker)
            
        start_idx=0

        if epoch_num in schedule:
            for param_group in optimizer_fed.param_groups:
                param_group['lr'] *= gamma
                print('New learnin rate ', param_group['lr'])

        optimizer_fed.zero_grad()

        model_grads=[]

        for i, param in enumerate(fed_model.parameters()):
            param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
            start_idx=start_idx+len(param.data.view(-1))
            param_=param_.cuda()
            model_grads.append(param_)

        optimizer_fed.step(model_grads)

        val_loss, val_acc = test(val_data_tensor,val_label_tensor,fed_model,criterion,use_cuda)
        te_loss, te_acc = test(te_data_tensor,te_label_tensor, fed_model, criterion, use_cuda)

        is_best = best_global_acc < val_acc

        best_global_acc = max(best_global_acc, val_acc)

        if is_best:
            best_global_te_acc = te_acc

        if epoch_num % 10 == 0:
            print('%s: at %s at_frac %.1f n_at %d n_mal_sel %d e %d fed_model val loss %.4f val acc %.4f best val_acc %f te_acc %f'%(aggregation, at_type, at_fraction, n_attacker, np.sum(krum_candidate < n_attacker), epoch_num, val_loss, val_acc, best_global_acc,best_global_te_acc))

        epoch_num+=1

==> Initializing global model
malicious grads shape  torch.Size([60, 848382])
bulyan: at our-agr at_frac 20.0 n_at 13 n_mal_sel 13 e 0 fed_model val loss 3.9942 val acc 4.8986 best val_acc 4.898579 te_acc 5.403110
bulyan: at our-agr at_frac 20.0 n_at 12 n_mal_sel 8 e 10 fed_model val loss 3.8320 val acc 4.8986 best val_acc 7.109761 te_acc 7.369749
bulyan: at our-agr at_frac 20.0 n_at 11 n_mal_sel 6 e 20 fed_model val loss 3.8638 val acc 5.4726 best val_acc 11.243822 te_acc 12.330107
bulyan: at our-agr at_frac 20.0 n_at 7 n_mal_sel 7 e 30 fed_model val loss 3.7695 val acc 22.5057 best val_acc 24.619028 te_acc 26.848229
bulyan: at our-agr at_frac 20.0 n_at 14 n_mal_sel 13 e 40 fed_model val loss 3.7401 val acc 24.0012 best val_acc 27.962829 te_acc 31.566619
bulyan: at our-agr at_frac 20.0 n_at 9 n_mal_sel 9 e 50 fed_model val loss 3.5776 val acc 30.2898 best val_acc 30.871602 te_acc 34.601524
bulyan: at our-agr at_frac 20.0 n_at 14 n_mal_sel 14 e 60 fed_model val loss 3.3096 val acc 28.8

bulyan: at our-agr at_frac 20.0 n_at 14 n_mal_sel 14 e 580 fed_model val loss 2.7058 val acc 37.1808 best val_acc 39.446046 te_acc 43.389621
bulyan: at our-agr at_frac 20.0 n_at 9 n_mal_sel 9 e 590 fed_model val loss 2.6828 val acc 38.2722 best val_acc 39.446046 te_acc 43.389621
bulyan: at our-agr at_frac 20.0 n_at 11 n_mal_sel 11 e 600 fed_model val loss 2.7482 val acc 37.5875 best val_acc 39.446046 te_acc 43.389621
bulyan: at our-agr at_frac 20.0 n_at 9 n_mal_sel 9 e 610 fed_model val loss 2.6022 val acc 39.4615 best val_acc 39.461491 te_acc 43.111614
bulyan: at our-agr at_frac 20.0 n_at 9 n_mal_sel 9 e 620 fed_model val loss 2.6053 val acc 39.5053 best val_acc 39.505251 te_acc 43.706240
bulyan: at our-agr at_frac 20.0 n_at 11 n_mal_sel 11 e 630 fed_model val loss 2.6795 val acc 38.3340 best val_acc 39.646829 te_acc 43.829798
bulyan: at our-agr at_frac 20.0 n_at 10 n_mal_sel 10 e 640 fed_model val loss 2.6583 val acc 37.7137 best val_acc 40.303233 te_acc 44.120675
bulyan: at our-agr 

bulyan: at our-agr at_frac 20.0 n_at 12 n_mal_sel 12 e 1170 fed_model val loss 2.3085 val acc 41.4719 best val_acc 44.486203 te_acc 47.896932
bulyan: at our-agr at_frac 20.0 n_at 13 n_mal_sel 13 e 1180 fed_model val loss 2.2591 val acc 42.6534 best val_acc 44.486203 te_acc 47.896932
bulyan: at our-agr at_frac 20.0 n_at 14 n_mal_sel 14 e 1190 fed_model val loss 2.3037 val acc 42.2544 best val_acc 44.486203 te_acc 47.896932
bulyan: at our-agr at_frac 20.0 n_at 11 n_mal_sel 11 e 1200 fed_model val loss 2.2158 val acc 44.0537 best val_acc 44.486203 te_acc 47.896932
bulyan: at our-agr at_frac 20.0 n_at 14 n_mal_sel 14 e 1210 fed_model val loss 2.3374 val acc 42.5813 best val_acc 44.486203 te_acc 47.896932
bulyan: at our-agr at_frac 20.0 n_at 13 n_mal_sel 13 e 1220 fed_model val loss 2.3202 val acc 41.7164 best val_acc 44.486203 te_acc 47.896932
bulyan: at our-agr at_frac 20.0 n_at 13 n_mal_sel 13 e 1230 fed_model val loss 2.2686 val acc 42.4552 best val_acc 44.486203 te_acc 47.896932
bulyan